In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install ultralytics

In [6]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [44]:
import cv2
import os
from ultralytics import YOLO
from IPython.display import Video, display


In [7]:
model = YOLO('yolov8n.pt')

In [14]:
!pip install -U ultralytics opencv-python tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is inc

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras import backend as K


In [16]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [45]:
INPUT_VIDEO = "/content/drive/MyDrive/walking video fottage/854100-hd_1920_1080_25fps.mp4"
OUTPUT_PATH = "/content/output_gender_tracking.mp4"
COMPRESSED_PATH = "/content/output_compressed.mp4" # Required for Colab display

In [46]:
yolo = YOLO("yolov8n.pt")  # nano = low RAM


In [47]:
cap = cv2.VideoCapture(INPUT_VIDEO)

In [48]:
cap = cv2.VideoCapture(INPUT_VIDEO)

if not cap.isOpened():
    print(f"Error: Could not open video file {INPUT_VIDEO}")
else:
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # We are resizing to 640x360, so the writer must match exactly
    TARGET_WIDTH = 640
    TARGET_HEIGHT = 360

    # Initialize VideoWriter
    # We use mp4v for writing, then convert to h264 later for display
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (TARGET_WIDTH, TARGET_HEIGHT))

    unique_person_ids = set()

    print("Processing video... this may take a moment.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize frame to match TARGET dimensions
        frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))

        # TRACKING
        results = model.track(
            frame,
            persist=True,
            classes=0, # 0 = person
            tracker="bytetrack.yaml",
            verbose=False
        )

        # DRAWING
        result = results[0]
        if result.boxes.id is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            ids = result.boxes.id.cpu().numpy()

            for box, track_id in zip(boxes, ids):
                unique_person_ids.add(int(track_id))
                x1, y1, x2, y2 = map(int, box)

                # Draw Box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Draw ID
                cv2.putText(frame, f"ID: {int(track_id)}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw Count
        cv2.putText(frame, f"Count: {len(unique_person_ids)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        out.write(frame)

Processing video... this may take a moment.


In [50]:
  cap.release()
  out.release()
  print("Processing complete.")

Processing complete.


In [60]:
import os

if os.path.exists(OUTPUT_PATH):
  print("Converting video for browser playback...")
  # Use ffmpeg to re-encode to H.264 so it plays in the browser
  os.system(f"ffmpeg -y -i {OUTPUT_PATH} -vcodec libx264 {COMPRESSED_PATH}")

  # Display the video
  display(Video(COMPRESSED_PATH, embed=True, width=640))
else:
  print("Error: Output file was not created.")

Converting video for browser playback...
